#### Importing libraries

In [1]:
import pandas as pd
import numpy as np

#### Importing the dataset

In [2]:
data = pd.read_csv('raw_mental_health_data.csv', low_memory=False)
data.head()

,index,Entity,Code,Year,Schizophrenia (%),Bipolar disorder (%),Eating disorders (%),Anxiety disorders (%),Drug use disorders (%),Depression (%),Alcohol use disorders (%)
0,0,Afghanistan,AFG,1990,0.16056,0.697779,0.101855,4.828830,1.677082,4.071831,0.672404
1,1,Afghanistan,AFG,1991,0.160312,0.697961,0.099313,4.829740,1.684746,4.079531,0.671768
2,2,Afghanistan,AFG,1992,0.160135,0.698107,0.096692,4.831108,1.694334,4.088358,0.670644
3,3,Afghanistan,AFG,1993,0.160037,0.698257,0.094336,4.830864,1.705320,4.096190,0.669738
4,4,Afghanistan,AFG,1994,0.160022,0.698469,0.092439,4.829423,1.716069,4.099582,0.669260


#### Upon examining the CSV file, I discovered it encompasses four tables, each with a unique structure. So I needed to extract these subtables from the raw data. 

In [3]:
# identify the indices that distinguish the various tables within the raw CSV data
data[data.Year == 'Year']

,index,Entity,Code,Year,Schizophrenia (%),Bipolar disorder (%),Eating disorders (%),Anxiety disorders (%),Drug use disorders (%),Depression (%),Alcohol use disorders (%)
6468,6468,Entity,Code,Year,Prevalence in males (%),Prevalence in females (%),Population,NaN,NaN,NaN,NaN
54276,54276,Entity,Code,Year,"Suicide rate (deaths per 100,000 individuals)",Depressive disorder rates (number suffering pe...,Population,NaN,NaN,NaN,NaN
102084,102084,Entity,Code,Year,Prevalence - Depressive disorders - Sex: Both ...,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# dividing the tables
table1 = data.iloc[:6468]

table2 = data.iloc[6469:54276]
table2.columns = data.iloc[6468]
table2 = table2.iloc[:,:7].drop(columns = 6468)

table3 = data.iloc[54277:102084]
table3.columns = data.iloc[54276]
table3 = table3.iloc[:,:7].drop(columns = 54276)

table4 = data.iloc[102085:]
table4.columns = data.iloc[102084]
table4 = table4.iloc[:,:5].drop(columns = 102084)

#### Extracted 4 subtables from the raw data

In [5]:
from IPython.display import display

display(table1.head())
display(table2.head())
display(table3.head())
display(table4.head())

,index,Entity,Code,Year,Schizophrenia (%),Bipolar disorder (%),Eating disorders (%),Anxiety disorders (%),Drug use disorders (%),Depression (%),Alcohol use disorders (%)
0,0,Afghanistan,AFG,1990,0.16056,0.697779,0.101855,4.828830,1.677082,4.071831,0.672404
1,1,Afghanistan,AFG,1991,0.160312,0.697961,0.099313,4.829740,1.684746,4.079531,0.671768
2,2,Afghanistan,AFG,1992,0.160135,0.698107,0.096692,4.831108,1.694334,4.088358,0.670644
3,3,Afghanistan,AFG,1993,0.160037,0.698257,0.094336,4.830864,1.705320,4.096190,0.669738
4,4,Afghanistan,AFG,1994,0.160022,0.698469,0.092439,4.829423,1.716069,4.099582,0.669260


6468,Entity,Code,Year,Prevalence in males (%),Prevalence in females (%),Population
6469,Afghanistan,AFG,1800,NaN,NaN,3280000
6470,Afghanistan,AFG,1801,NaN,NaN,3280000
6471,Afghanistan,AFG,1802,NaN,NaN,3280000
6472,Afghanistan,AFG,1803,NaN,NaN,3280000
6473,Afghanistan,AFG,1804,NaN,NaN,3280000


54276,Entity,Code,Year,"Suicide rate (deaths per 100,000 individuals)","Depressive disorder rates (number suffering per 100,000)",Population
54277,Afghanistan,AFG,1800,NaN,NaN,3280000
54278,Afghanistan,AFG,1801,NaN,NaN,3280000
54279,Afghanistan,AFG,1802,NaN,NaN,3280000
54280,Afghanistan,AFG,1803,NaN,NaN,3280000
54281,Afghanistan,AFG,1804,NaN,NaN,3280000


102084,Entity,Code,Year,Prevalence - Depressive disorders - Sex: Both - Age: All Ages (Number) (people suffering from depression)
102085,Afghanistan,AFG,1990,318435.81367
102086,Afghanistan,AFG,1991,329044.773956
102087,Afghanistan,AFG,1992,382544.572895
102088,Afghanistan,AFG,1993,440381.507393
102089,Afghanistan,AFG,1994,456916.645489


#### Merging 4 subtables together 

In [6]:
# filter out years between 1990-2017 and countries from table1

table1 = data.iloc[:6468].copy()
table1['Year'] = pd.to_numeric(table1['Year'], errors='coerce')

filtered_table1 = table1[(table1['Year'] >= 1990) & (table1['Year'] <= 2017)]

relevant_years = filtered_table1['Year'].unique()
relevant_entities = filtered_table1['Entity'].unique()

# filter out all 4 tables
def filter_table_by_criteria(table, years, entities):
    table['Year'] = pd.to_numeric(table['Year'], errors='coerce')
    return table[(table['Year'].isin(years)) & (table['Entity'].isin(entities))]

table2_filtered = filter_table_by_criteria(table2.copy(), relevant_years, relevant_entities)
table3_filtered = filter_table_by_criteria(table3.copy(), relevant_years, relevant_entities)
table4_filtered = filter_table_by_criteria(table4.copy(), relevant_years, relevant_entities)

In [7]:
# Convert 'Year' to integer before merging
table1['Year'] = pd.to_numeric(table1['Year'], errors='coerce').astype(int)
table2_filtered['Year'] = pd.to_numeric(table2_filtered['Year'], errors='coerce').astype(int)
table3_filtered['Year'] = pd.to_numeric(table3_filtered['Year'], errors='coerce').astype(int)
table4_filtered['Year'] = pd.to_numeric(table4_filtered['Year'], errors='coerce').astype(int)


In [8]:
# merging all 4 tables together based on key columns
from functools import reduce
import pandas as pd

tables_to_merge = [table1, table2_filtered, table3_filtered, table4_filtered]
merged_dataset = reduce(lambda left, right: pd.merge(left, right, on = ['Year', 'Code', 'Entity'], how = 'outer'), tables_to_merge)

In [9]:
merged_dataset

,index,Entity,Code,Year,Schizophrenia (%),Bipolar disorder (%),Eating disorders (%),Anxiety disorders (%),Drug use disorders (%),Depression (%),Alcohol use disorders (%),Prevalence in males (%),Prevalence in females (%),Population_x,"Suicide rate (deaths per 100,000 individuals)","Depressive disorder rates (number suffering per 100,000)",Population_y,Prevalence - Depressive disorders - Sex: Both - Age: All Ages (Number) (people suffering from depression)
0,0,Afghanistan,AFG,1990,0.16056,0.697779,0.101855,4.828830,1.677082,4.071831,0.672404,3.499982,4.647815,12412000.000000,10.318504,4039.755763,12412000.000000,318435.81367
1,1,Afghanistan,AFG,1991,0.160312,0.697961,0.099313,4.829740,1.684746,4.079531,0.671768,3.503947,4.655772,13299000.000000,10.32701,4046.256034,13299000.000000,329044.773956
2,2,Afghanistan,AFG,1992,0.160135,0.698107,0.096692,4.831108,1.694334,4.088358,0.670644,3.508912,4.662066,14486000.000000,10.271411,4053.709902,14486000.000000,382544.572895
3,3,Afghanistan,AFG,1993,0.160037,0.698257,0.094336,4.830864,1.705320,4.096190,0.669738,3.513429,4.669012,15817000.000000,10.376123,4060.203474,15817000.000000,440381.507393
4,4,Afghanistan,AFG,1994,0.160022,0.698469,0.092439,4.829423,1.716069,4.099582,0.669260,3.515578,4.67305,17076000.000000,10.575915,4062.290365,17076000.000000,456916.645489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6463,6463,Zimbabwe,ZWE,2013,0.15567,0.607993,0.117248,3.090168,0.766280,3.128192,1.515641,2.769193,3.424106,13350000.000000,28.3612,3048.264249,13350000.0,303564.60359
6464,6464,Zimbabwe,ZWE,2014,0.155993,0.60861,0.118073,3.093964,0.768914,3.140290,1.515470,2.778101,3.437674,13587000.000000,27.605547,3056.996704,13587000.0,311665.769283
6465,6465,Zimbabwe,ZWE,2015,0.156465,0.609363,0.11947,3.098687,0.771802,3.155710,1.514751,2.789152,3.455323,13815000.000000,27.197061,3068.250731,13815000.0,320638.507158
6466,6466,Zimbabwe,ZWE,2016,0.157111,0.610234,0.121456,3.104294,0.772275,3.174134,1.513269,2.799308,3.479071,14030000.000000,26.839591,3081.782858,14030000.0,330437.353798


#### Creating a new CSV file from the merged tables

In [10]:
merged_dataset.to_csv('C:/Users/Benji/python_learn/global_mental_disorders/merged_dataset.csv', index=False)